In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [5]:
dataset = fetch_20newsgroups(data_home='D:/code python/LDA2vec/bangla news',subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [6]:
# number of documents
len(docs)

22674

In [7]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [8]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

  0%|▏                                                                            | 47/22674 [00:09<1:19:44,  4.73it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|████████████████████████████████████████████████████████████████████████████| 22674/22674 [43:20<00:00,  8.72it/s]


number of removed short documents: 171
total number of tokens: 3728284
number of tokens to be removed: 1257407
number of additionally removed short documents: 217
total number of tokens: 2468364

minimum word count number: 1
this number can be less than MIN_COUNTS because of document removal


In [9]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [10]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

22286it [00:09, 2407.97it/s]


In [11]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [12]:
# number of windows (equals to the total number of tokens)
data.shape[0]

2468364

# Get unigram distribution

In [13]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [14]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

KeyError: "word '19028' not in vocabulary"

In [15]:
# number of unique words
vocab_size

19029

# Prepare initialization for document weights

In [16]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [18]:
%%time
n_topics = 9
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 30.7 s


In [19]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : পুরস্কার দেশটির চলচ্চিত্র ওবামা খবরে উৎসবের উৎসব এএফপির দিল্লির রাজধানী
topic 1 : লাশ ছাত্রলীগের ঘটনায় স্থানীয় শহরের দুর্বৃত্তরা মহানগর মিছিল হামলা ভর্তি
topic 2 : পরিমাণ মন্ত্রণালয় প্রকল্প ডলারের বিদ্যুৎ উন্নয়ন শিল্প বন্দর চলতি কমে
topic 3 : ধারাবাহিক অনুষ্ঠান সরাসরি বিকাল ০৭:০০- নিউজ গল্প ১২:০০- ছবিতে সিনেমা
topic 4 : মোবাইল গুগল অ্যাপল প্রযুক্তি অধ্যাপক স্মার্টফোন অ্যাপলের স্যামসাং ইঞ্চি ফোন
topic 5 : নির্বাচনের সংসদ জিয়া মহাসচিব সাংসদ বিরোধী আন্দোলন নির্বাচনে মন্ত্রী হাসিনা
topic 6 : মিনিটে টিভি বাংলা ৯-৩০ গোল মেলায় মেলা কুমার পেঁয়াজের স্টার
topic 7 : তদন্ত আদালত অভিযান নির্দেশ আদালতে মামলার সিটি এলাকা সচিব গাছ
topic 8 : উইকেট বিশ্বকাপ দুর্ঘটনা ম্যাচে উইকেটে বিশ্বকাপে ক্রিকেট বিশ্বকাপের কাল রানে


In [20]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

 21%|███████████████▋                                                            | 4592/22286 [00:09<00:38, 459.91it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|███████████████████████████████████████████████████████████████████████████| 22286/22286 [00:22<00:00, 992.79it/s]


# Save data

In [21]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)